# Simulated Quantum Noise Spectroscopy

In this notebook we perform a series of simulated quantum noise spectroscopy experiment. The single-qubit semi-classical noisy Hamiltonian model is

$$H(t) = \eta\sigma_z+\Omega(t)\sigma_x\,.$$

We simulate this via a discrete SchWARMA model with discrete time Hamiltonian

$$H_k = y_k\sigma_z+\Omega_k\sigma_x\,.$$

The qubit is prepared and measured in the $|+\rangle$ state, and a series of $X$ $\pi$-pulse control sequences are applied, recording the survival probabilities.  These are averaged and the filter function formalism is used to perform a linear inversion to obtain an estimated spectrum.

In [ ]:
%matplotlib inline

import numpy as np
import scipy.signal as si
import scipy.linalg as la
import matplotlib.pyplot as pl
import scipy.optimize as op
import mezze.channel as ch

In [ ]:
# Routine for performing simulated QNS experiment
def schcpmg_recon(b, b_back, freqs, N, a=[1], a_back=[1], num_MC = 50):
    # b and a are the underlying ARMA coefficients for the SchWARMA dephasing model
    # b_back and a_back are the underlying ARMA coefficients for a second, independent dephasing SchWARMA model (not used in the paper)
    # freqs are discrete time frequences of the modulation functions used to construct the sequences, with N the their length (in gate time)
    # num_MC is the number of monte carlo runs to perform
    
    #Set up X, X/2 and -X/2 gates
    X = ch.QuantumChannel(ch._sigmaX,'unitary').kraus()[0]
    X2 = ch.QuantumChannel(la.expm(-1j*ch._sigmaX*np.pi/4.),'unitary').kraus()[0]
    nX2 = ch.QuantumChannel(la.expm(1j*ch._sigmaX*np.pi/4.),'unitary').kraus()[0]
    
    flips = []
    ps = np.zeros((len(freqs),num_MC))

    mod_funs = []
    for i, ww in enumerate(freqs):
        
        #define the modulation function of the control sequence defined by freq
        mod_fun = np.sign(np.cos(ww*np.arange(1,N+1)))
        mod_fun[mod_fun==0]=1
        mod_fun = (mod_fun+1)/2
        mod_funs.append(mod_fun)
        
        #determine when the modulation function changes sign -- defines when an X gate is applied
        flip = np.concatenate(([0],np.abs(mod_fun[1:]-mod_fun[:-1])))
        flips.append(flip)
        
        for j in range(num_MC):
            
            #compute noisy sequence
            ys = si.lfilter(b,a,np.random.randn(N+len(b)))[len(b):]
            ys_back = si.lfilter(b_back,a_back,np.random.randn(N+len(b_back)))[len(b_back):]
            etas = ys+ys_back#np.exp(1j*(ys+ys_back))
            
            #prepare sate in |+>
            rho = X2.dot(np.eye(2,dtype=np.complex)[:,0,np.newaxis])
            for eta, f in zip(etas,flip):
                if f == 1: #An X gate+noise is applied
                    eta = np.exp(1j*eta)
                    rho = X.dot(np.diag([eta,eta.conj()]).dot(rho))
                    #rho = la.expm(1j*np.pi/2*ch._sigmaX+1j*eta*ch._sigmaZ).dot(rho)
                else: #An identity gate+noise
                    eta = np.exp(1j*eta)
                    rho = np.diag([eta,eta.conj()]).dot(rho)
            
            #rotate back to |0>
            if np.sum(flip)%2==0:
                rho = nX2.dot(rho)
            else:
                rho = X2.dot(rho)
            
            #measure in |0> -- effectively measuring in $|+>
            if True:#np.sum(flips)%2 ==0:
                ps[i,j] = np.abs(rho[0,0])**2
            else:
                ps[i,j] = np.abs(rho[1,1])**2
    return ps

## Lowpass and Bandpass Noise
This simulates noise that has flat bands of power

In [ ]:
num_MC = 500
N = 128
worN = 2**16

#Want evenly spaced modulation frequencies
freqs = np.linspace(0,np.pi,N//2+1)[:-1]

mod_funs=[]
for i, ww in enumerate(freqs):
    mod_fun = np.sign(np.cos(ww*np.arange(1,N+1)))
    mod_fun[mod_fun==0]=1
    mod_funs.append(mod_fun)

s_pow = .02#625

#Low pass noise
b1 = si.firwin(N, .1)
b1 = b1/la.norm(b1)*s_pow
b_back = np.array([.1*s_pow])

w1, h1 = si.freqz(b1,[1],worN=N//2)
w1F, h1F = si.freqz(b1,[1],worN=worN,whole=True)

#Band pass noise
b2 = si.firwin(N, .2)
b2*=np.cos(.37*np.pi*np.arange(len(b2)))
b2 = b2/la.norm(b2)*s_pow

w2, h2 = si.freqz(b2,[1],worN=N//2)
w2F, h2F = si.freqz(b2,[1],worN=worN,whole=True)

In [ ]:
ps1 = schcpmg_recon(b1, b_back,freqs,N,num_MC=num_MC)
ps2 = schcpmg_recon(b2, b_back,freqs,N,num_MC=num_MC)

In [ ]:
#Construct a reconstruction matrix from the filter functions
PhiRecon = np.array([np.abs(np.fft.fft(mf,N))**2/(.5*N) for mf in mod_funs])
PhiRecon = PhiRecon[:,:N//2]
PhiRecon[1:,:] = PhiRecon[1:,:]*2

#Convert survival probabilites to chi and invert using NNLS
chis1 = -np.log(2*np.mean(ps1,1)-1)
Snn1 = op.nnls(PhiRecon,chis1)[0]
Snn1[Snn1==0] = np.min(Snn1[Snn1>0])

chis2 = -np.log(2*np.mean(ps2,1)-1)
Snn2 = op.nnls(PhiRecon,chis2)[0]
Snn2[Snn2==0] = np.min(Snn2[Snn2>0])

## Multi-pole spectra
This simulates noise that is "peaky" at a few harmonics

In [ ]:
#Defines which frequencies the peaks will be at
freq_idxs = [10, 17, 22]
poles  =[]
for idx in freq_idxs:
    poles+=[np.exp(1j*freqs[idx]),np.exp(-1j*freqs[idx])]

#Convert from poles to ARMA coeffcients
b_mp,a_mp = si.zpk2tf([],poles,s_pow/60)
w_mp, h_mp = si.freqz(b_mp,a_mp,worN=N)

ps_mp= schcpmg_recon(b_mp,5*b_back,freqs,N,a=a_mp,num_MC=num_MC)

In [ ]:
#Estimate noise from ps
chis_mp= -np.log(2*np.mean(ps_mp,1)-1)
Snn_mp = op.nnls(PhiRecon,chis_mp)[0]
Snn_mp[Snn_mp==0] = np.min(Snn_mp[Snn_mp>0])

In [ ]:
fig, (ax1,ax2) = pl.subplots(2,1)

ax1.plot(freqs,10*np.log10(Snn1),'+')
ax1.plot(freqs,10*np.log10(Snn2),'x')
ax1.set_prop_cycle(None)
ax1.plot(w1, 20*np.log10(np.abs(h1)))
ax1.plot(w2, 20*np.log10(np.abs(h2)))

ax1.set_ylim([-80,-20])

ax1.legend(['Low Pass','Band Pass'],fontsize=14,loc='lower right',ncol=2)
ax1.set_title('Bandlimited PSD Reconstructions',fontsize=16)
ax1.grid()
ax1.set_xticklabels([])
ax2.plot(freqs,10*np.log10(Snn_mp),'-r+')
for idx in freq_idxs:
    ax2.axvline(freqs[idx],linestyle='--',color = 'k')
ax2.grid()
ax2.set_title('Multi-Pole PSD Reconstruction',fontsize=16)
fig.text(0.5, 0.02, 'Normalized Frequency', ha='center',fontsize=14)
fig.text(0.01, 0.5, 'Power, dB', va='center', rotation='vertical',fontsize=14)

ax2.legend(['PSD Estimate','True Poles'], fontsize=14,loc='upper right')
pl.subplots_adjust(hspace=.3)
pl.savefig('schwarma_psd_recon_bl_mt.eps',bbox_inches='tight')

pl.show()

## $f^{-\alpha}$ spectra
This uses the method of S. Plaszczynski, Fluctuation and Noise Letters7, R1 (2007) to define ARMA models for $f^{-\alpha}$ noise

In [ ]:
#In order to see 1/f noise effectively, we require a much better frequency resolution
#which implies longer sequences, and thus overall weaker noise
N_pink = 2048
freqs_pink = np.linspace(0,np.pi,N_pink//2+1)[:-1]

mod_funs_pink=[]
for i, ww in enumerate(freqs_pink):
    mod_fun = np.sign(np.cos(ww*np.arange(1,N_pink+1)))
    mod_fun[mod_fun==0]=1
    mod_funs_pink.append(mod_fun)
    
PhiRecon_pink = np.array([np.abs(np.fft.fft(mf,N_pink))**2/(.5*N_pink) for mf in mod_funs_pink])
PhiRecon_pink = PhiRecon_pink[:,:N_pink//2]
PhiRecon_pink[1:,:] = PhiRecon_pink[1:,:]*2

alphas = [.5,1.,1.5,2]

#This uses the method of S. Plaszczynski, Fluctuation and Noise Letters7, R1 (2007) for defining ARMA models
Snn_p =[]
ps_p = []
Phi_p = []
chi_p = []
w_p =[]
h_p = []

as_p =[]
bs_p = []
for alpha in alphas:

    wl = .001*np.pi
    wh = .999*np.pi
    Nf = np.ceil(2.5*(np.log10(wh)-np.log10(wl)))
    delp = (np.log10(wh)-np.log10(wl))/Nf
    logps = np.log10(wl)+.5*(1-alpha/2.)*delp + np.arange(Nf)*delp
    logzs = logps+alpha/2.*delp
    ps = 10**(logps)
    zs = 10**(logzs)

    pstx = (1-ps)/(1+ps)
    zstx = (1-zs)/(1+zs)
    bb,aa = si.zpk2tf(zstx,pstx,k=1e-4)
    w_pa,h_pa = si.freqz(bb,aa,worN=N_pink)
    
    as_p.append(aa)
    bs_p.append(bb)
    #b_p2 = np.array([(1.+r1)/(1.+r0),-(1.-r1)/(1+r0)])*.00001
    #a_p2 = np.array([1,-(1-r0)/(1+r0)])

    ww, hh = si.freqz(bb,aa,worN=N_pink/2)
    w_p.append(ww)
    h_p.append(hh)
    
    ps_temp = schcpmg_recon(bb,0*b_back,freqs_pink,N_pink,a=aa,num_MC=num_MC)
    
    chis_temp= -np.log(2*np.mean(ps_temp,1)-1)
    Snn_temp = op.nnls(PhiRecon_pink,chis_temp)[0]
    Snn_temp[Snn_temp==0] = np.min(Snn_temp[Snn_temp>0])
    
    Snn_p.append(Snn_temp)
    #Phi_p.append(Phi_temp)
    #chi_p.append(chi_temp)
    #ps_p.append(ps_temp)

In [ ]:
# Find the best fits for lines to show in log-log plots
coeffs = np.zeros(len(alphas))
for i,alpha in enumerate(alphas):
    idx = (np.log10(freqs_pink)>-2) & (np.log10(freqs_pink) <0.0)
    fun = lambda x: np.log10(x/(freqs_pink[idx]**alpha))-np.log10(Snn_p[i][idx]+1e-9)
    out = op.least_squares(fun,1e-6)
    coeffs[i] = out['x']
    print(out['x'])

In [ ]:
pl.figure()
for i in range(0,len(alphas)):
    pl.plot(np.log10(freqs_pink[1:]),10*np.log10(Snn_p[i][1:]),'+',alpha=0.55)
#pl.plot(np.log10(w_2p[1:]/np.pi), 20*np.log10(np.abs(h_p2[1:])/np.sqrt(N_pink)))
#pl.gca().set_color_cycle(None)
pl.gca().set_prop_cycle(None)
for i in range(len(alphas)):
    pl.plot(np.log10(freqs_pink[1:]),10*np.log10(coeffs[i]/freqs_pink[1:]**alphas[i]),'--k')
pl.grid()
pl.xlabel('Normalized Frequency',fontsize=14)
pl.ylabel('Power',fontsize=14)
pl.legend([r'$\alpha=0.5$',r'$\alpha=1$',r'$\alpha=1.5$',r'$\alpha=2$'],fontsize=14)
pl.title(r'$f^{-\alpha}$ PSD Reconstructions',fontsize=16)
#pl.savefig('schwarma_psd_recon_finv.eps',bbox_inches='tight')
pl.show()